# An effort to write GSLR out of the OmicsIntegrator Framework

In [1]:
%pylab inline
import sys
import pickle
import pandas as pd

from sklearn.preprocessing import LabelEncoder

from matplotlib_venn import venn3, venn3_circles, venn2

sys.path.append('/Users/alex/Documents/OmicsIntegrator2/src/')
from graph import Graph

Populating the interactive namespace from numpy and matplotlib


### I. Load Synthetic Dataset which is known to be at least somewhat connected in the interactome. 

In [2]:
dataset = pd.read_csv('/Users/alex/Documents/proteomics/synthetic_proteomics/generated_data/ludwig_svd_normals/hsa04110_inbiomap_exp.csv', index_col=0)
dataset.head()

,ZNF91,NDEL1,ELAVL1,SUMO1,SUMO3,CHMP5,UBC,HTT,E2F4,ACP5,...,SPANXN4,ZNF605,SERPINB10,ANKAR,RRH,DHH,CYSLTR1,ZNF268,COL23A1,MEDAG
hsa04110,0.0,0.014856,0.264810,0.109797,-0.976005,0.173769,0.0,0.285756,0.288839,-0.231566,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
negative,0.0,-0.228736,0.089442,0.339065,-0.597546,0.040049,0.0,-0.098450,0.073976,-0.494193,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
negative,-0.0,-0.715740,-0.143526,-0.480106,0.125846,0.284645,-0.0,0.021132,-0.115910,-0.016950,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
negative,-0.0,-0.383979,0.359009,0.394243,-0.387827,-0.281805,0.0,-0.922308,0.340674,-0.278771,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
negative,0.0,-0.023037,0.196800,-0.292508,0.165186,0.105207,0.0,-0.521385,-0.194466,-0.234121,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### II. Extract Labels 

In [3]:
labeler = LabelEncoder()
labeler.fit(dataset.index.tolist())
labels = labeler.transform(dataset.index.tolist())
y = labels
y

array([0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0])

### III. Graph Sparse Logistic Regression

In [6]:
interactome_file = '../pcsf/inbiomap_temp.tsv'
graph = Graph(interactome_file, {'a':1500, 'b':4, 'w':7})

08:12:40 - Graph: INFO - Duplicated edges in the interactome file (we'll keep the max cost):
08:12:40 - Graph: INFO - []


In [ ]:
def gslr(graph, X, y, learning_rate, iterations):

    theta = np.zeros(len(graph.nodes))

    for i in range(iterations):

        gradients = gradient_descent(X.transpose(), theta, y)

        theta = theta - (learning_rate * gradients)

        vertex_indices, edge_indices = graph.pcsf(theta)

        theta[~vertex_indices] = 0

    return theta


def gradient_descent(X, theta, y):

    denominators = 1.0 + np.exp(np.multiply(y, np.dot(X, theta)))
    return -np.sum(np.divide(np.multiply(np.expand_dims(y, axis=1), X), np.expand_dims(denominators, axis=1)), axis=0)

